# Across scenario basic analysis

### Load packages

In [1]:
import os
import sys
import urllib, io
os.getcwd()
sys.path.append("..")
sys.path.append("../utils")
sys.path.append("../analysis/utils")


import numpy as np
import scipy.stats as stats
import pandas as pd

import pymongo as pm
from collections import Counter
import json
import re
import ast

from PIL import Image, ImageOps, ImageDraw, ImageFont 

from io import BytesIO
import base64

from tqdm.notebook import tqdm

import  matplotlib
from matplotlib import pylab, mlab, pyplot
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
plt.style.use('seaborn-white')

import seaborn as sns
sns.set_context('talk')
sns.set_style('darkgrid')
%matplotlib inline

import scipy.stats
import random

from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

In [2]:
#display all columns
pd.set_option('display.max_columns', None)

## Theming

In [3]:
sns.set_style("white")

### Helper functions

In [4]:
#helper function for pd.agg
def item(x):
    """Returns representative single item"""
    return x.tail(1).item()

### Set up directory paths to plots and data

In [5]:
## directory & file hierarchy
proj_dir = os.path.abspath('..')
datavol_dir = os.path.join(proj_dir,'data')
analysis_dir =  os.path.abspath('.')
results_dir = os.path.join(proj_dir,'results')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
json_dir = os.path.join(results_dir,'json')
exp_dir = os.path.abspath(os.path.join(proj_dir,'behavioral_experiments'))
png_dir = os.path.abspath(os.path.join(datavol_dir,'png'))

## add helpers to python path
if os.path.join(proj_dir,'stimuli') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'stimuli'))
    
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)   
    
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)       
    
## add helpers to python path
if os.path.join(proj_dir,'utils') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'utils'))   

def make_dir_if_not_exists(dir_name):   
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
    return dir_name

## create directories that don't already exist        
result = [make_dir_if_not_exists(x) for x in [results_dir,plot_dir,csv_dir]]

## Load in data

Assumes exported csvs from `basic_analysis.ipynb` in results folder

In [6]:
studies = [
    "drop_pilot",
    "collision_pilot",
    "rollingsliding_pilot",
    "dominoes_pilot",
    "clothiness_pilot",
    "towers_pilot",
    "linking_pilot",
    "containment_pilot",
]

In [22]:
ls `echo $csv_dir/'humans'`

human_accuracy-clothiness_pilot-production_2_testing.csv
human_accuracy-collision_pilot-production_2_testing.csv
human_accuracy-containment_pilot-production_2_testing.csv
human_accuracy-dominoes_pilot-production_1_testing.csv
human_accuracy-drop_pilot-production_2_testing.csv
human_accuracy-linking_pilot-production_2_testing.csv
human_accuracy-rollingsliding_pilot-production_2_testing.csv
human_accuracy-towers_pilot-production_2_testing.csv
human_responses-clothiness_pilot-production_2_testing.csv
human_responses-collision_pilot-production_2_testing.csv
human_responses-containment_pilot-production_2_testing.csv
human_responses-dominoes_pilot-production_1_testing.csv
human_responses-drop_pilot-production_2_testing.csv
human_responses-linking_pilot-production_2_testing.csv
human_responses-rollingsliding_pilot-production_2_testing.csv
human_responses-towers_pilot-production_2_testing.csv


In [45]:
#load all experiments as one dataframe
prefix = "human_responses-"
suffix = lambda s: "-production_%d_testing" % (1 if 'dominoes' in s else 2)
make_fname = lambda s: prefix + s + suffix(s) + ".csv"
df = []
for l in studies:
    _df = pd.read_csv(os.path.join(csv_dir, 'humans', make_fname(l)))
    if 'study' not in _df.columns:
        _df = _df.assign(study=[l] * len(_df), axis=0)
    df.append(_df)
df = pd.concat(df)
# df = pd.concat([pd.read_csv(os.path.join(csv_dir, 'humans', make_fname(l))) for l in studies])
print("Loaded dataframes")

Loaded dataframes


In [46]:
df['study']

0               drop_pilot
1               drop_pilot
2               drop_pilot
3               drop_pilot
4               drop_pilot
               ...        
15145    containment_pilot
15146    containment_pilot
15147    containment_pilot
15148    containment_pilot
15149    containment_pilot
Name: study, Length: 120450, dtype: object

In [47]:
#save nice scenario name
df['scenario'] = df['study'].apply(lambda x: x.split('_')[0])

In [48]:
#sort by it
df.sort_values('scenario',inplace=True)

In [49]:
#subtract stimulus presentation time from reaction time
df['rt'] = df['rt'] - 2500

In [50]:
#drop reaction times longer than 10 seconds
df['rt'] = df['rt'].apply(lambda x: x if x < 10000 else np.nan)

In [51]:
def outcome_helper(correct,response):
    response = response == "YES"
    if correct and response: return "hit"
    if not correct and response: return "false_alarm"
    if correct and not response: return "correct_rejection"
    if not correct and not response: return "miss"

In [52]:
#encode response kind
df['outcome'] = [outcome_helper(correct, response) for correct, response in zip(df['correct'],df['response'])]

In [53]:
df['outcome'].value_counts()

hit                  46011
correct_rejection    43379
false_alarm          16846
miss                 14214
Name: outcome, dtype: int64

## Plots

### Per stimulus

In [ ]:
df['response'] = df['response'] == "YES" #encode response as boolean

In [ ]:
per_stim_agg = df.groupby('stim_ID').agg({
    'scenario' : lambda s: s.head(1),
    'correct' : lambda cs: np.mean([1 if c == True else 0 for c in cs]),
    'response' : 'mean',
    'c' : 'count',
    'stim_url' : lambda s: s.head(1),
})
per_stim_agg.sort_values('scenario',inplace=True)

In [ ]:
g = sns.FacetGrid(per_stim_agg, col="scenario", height=6)
g.map(sns.histplot, "correct", bins=40, binrange=[0,1])
g.set_axis_labels("Correct/Total per Stimulus","Count")
g.savefig(os.path.join(plot_dir,"per_stim_16.pdf"))

Individual plots

In [ ]:
scenario = "dominoes"

g = sns.FacetGrid(per_stim_agg.query("scenario == @scenario"), col="scenario", height=2, aspect=2)
g.map(sns.histplot, "correct", bins=40, binrange=[0,1])
g.set_axis_labels("Correct/Total per Stimulus","Count")
g.savefig(os.path.join(plot_dir,"per_stim_"+scenario+".pdf"))

In [ ]:
per_stim_agg.query("scenario == @scenario")

### Per person

do people have a bias for yes or no?

In [ ]:
per_person_agg = df.groupby('gameID').agg({
    'scenario' : lambda s: s.head(1),
    'correct' : lambda cs: np.mean([1 if c == True else 0 for c in cs]),
    'response' : 'mean',
    'c' : 'count',
})
per_person_agg.sort_values('scenario',inplace=True)

What is the mean correctness for the guesses of a single participant?

In [ ]:
g = sns.FacetGrid(per_person_agg, col="scenario", height=6)
g.map(sns.histplot, "correct", bins=40, binrange=[0,1])
g.set_axis_labels("Correct/Total per Participant","Count")
g.savefig(os.path.join(plot_dir,"per_subject_16.pdf"))

What is the Yes/No bias for each person? 1 corresponds to Yes. Perfect guesses every time would mean a response of 0.5

In [ ]:
g = sns.FacetGrid(per_person_agg, col="scenario", height=6)
g.map(sns.histplot, "response", bins=40, binrange=[0,1])

In [ ]:
df.groupby('scenario').agg({'response':'mean'})

### Per guess

What is the distribution of reaction times? This does not include the 2500ms it takes for the stimulus to be shown, where the buttons become available only afterwards

In [ ]:
g = sns.FacetGrid(df, col="scenario", hue="correct", height=6)
g.map(sns.histplot, "rt", bins=40, binrange=[0,10000])
g.add_legend()

In [ ]:
g = sns.FacetGrid(df, col="scenario", hue="correct", height=6)
g.map(sns.violinplot, "correct","rt", order=[True,False])

In [ ]:
g = sns.FacetGrid(df, col="scenario", hue="outcome", height=6)
g.map(sns.violinplot, "outcome","rt", order=["hit","correct_rejection","miss","false_alarm"])

### General

What is the d-prime of the experiment?

### Easy & adversarial example cases
> Easy, hard, chance, adversarial; reliable, noisy

In [ ]:
num_stims = 3

for scenario in per_stim_agg['scenario'].unique():
    print("*****",scenario,"*****")
    df_s = per_stim_agg[per_stim_agg['scenario'] == scenario]
    df_s = df_s.sort_values('correct')
    #get easy stims
    selection = df_s[df_s['correct']>0.90].sample(num_stims)
    print("---- Easy stims ----")
    display(selection)
    print(list(selection['stim_url']),"\n")
    #get hard stims
    selection = df_s[(df_s['correct']>0.70) & (df_s['correct']<0.80)].sample(num_stims)
    print("---- Hard stims ----")
    display(selection)
    print(list(selection['stim_url']),"\n")
    #get chance stims
    selection = df_s[(df_s['correct']>0.45) & (df_s['correct']<0.55)].sample(num_stims)
    print("---- Chance stims ----")
    display(selection)
    print(list(selection['stim_url']),"\n")
    #get adversarial stims
#     selection = df_s[df_s['correct']<0.20].sample(num_stims)
    selection = df_s.head(num_stims)
    print("---- Adversarial stims ----")
    display(selection)
    print(list(selection['stim_url']),"\n")

In [ ]:
df['occluders'].iloc[3]

## Saving out relevant data

In [ ]:
pd.DataFrame(df['sessionID'].unique()).to_csv(os.path.join(csv_dir,"sessionIDs.csv"))